In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import string
from string import punctuation, digits
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/reddit-vaccine-myths/reddit_vm.csv')

In [ ]:
df.head()

In [ ]:
df['score'].unique()

In [ ]:
#title with high score
print(df[df['score']==1187]['title'])

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(df['comms_num'])

In [ ]:
#high score and high comments
df[df['comms_num']==596]

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(df['score'])

In [ ]:
#considering title without 'comment' or 'Comment'
title = df[(df['title'] != 'comment') & (df['title'] != 'Comment')]['title']
    

In [ ]:
title

In [ ]:
words = ''

#processing the title
for val in title:
        
        #remove numbers
        rm_num = ''.join(char for char in val if not char.isdigit())
        #remove puntuations
        rm_pun = ''.join(char for char in rm_num if char not in punctuation)
        
        # split the value 
        tokens = rm_pun.split() 
    
        # Converts each token into lowercase 
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower() 
    
        #exclude the extra words
        words += " ".join(word for word in tokens if word not in stopwords.words('english'))+" "

In [ ]:
#Building wordcloud
stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', colormap = 'rainbow',
                stopwords = stopwords, 
                min_font_size = 10).generate(words)

In [ ]:
plt.figure(figsize = (10,10), facecolor = 'red') 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
df = df[(df['title'] != 'comment') & (df['title'] != 'Comment')]

In [ ]:
#Sentiment Analyser
sa = SentimentIntensityAnalyzer()
df['scores'] = df['title'].apply(lambda text: sa.polarity_scores(text))


In [ ]:
df[['scores','title']]

In [ ]:
#computing the compound score
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])     

In [ ]:
def compute_score(c):
    if c > 0.0:
        return 'positive'
    elif c == 0:
        return 'neutral'
    else:
        return 'negative'
df['compound_score'] = df['compound'].apply(lambda c :compute_score(c))

In [ ]:
df[['scores','title','compound','compound_score']]

In [ ]:
plt.figure(figsize = (8,8)) 
sns.countplot(df['compound_score'],palette = 'coolwarm')